# 第11章 データベースのマイグレーション

## 11.1 概要

### Djangoの「マイグレーション」
設定ファイルとモデルの定義からデータベースを作成したり定義を変更したりするためのコマンドを実行すること

#### Djangoのマイグレーションコマンド

| コマンド | 説明 |
| :------- | :--- |
| makemigrations | モデルの変更差分をチェックしてマイグレーションファイルを作成 |
| migrate | 設定ファイルで定義したデータベースに、マイグレーションファイルの内容を反映してテーブルを作成・変更 |

## 11.2 makemigrations（マイグレーションファイルの作成）
- 自作したアプリケーションのモデルの変更差分からマイグレーションファイルを作成する
- 引数なしで実行すると、設定ファイルの`INSTALLED_APPS`に登録された全てのアプリケーションに対して処理が行われる
- 個別のアプリケーションに限定するためには、アプリケーション名を引数に設定する。<br />複数のアプリケーション名をスペース区切りで列挙することも可能

#### マイグレーションファイル
- 各アプリケーションディレクトリの`migrations`パッケージ内に「<4桁の数字>\_<自動生成されたマイグレーション名>.py」という名前で作成される
  - 4桁の数字はmigrationパッケージ配下のマイグレーションファイルの状況に応じて自動でインクリメントされる
  - 自動生成されたマイグレーション名を任意の名前に変更したい場合は、`makemigrations`コマンドの`--name`オプションにマイグレーション名を渡す

## 11.3 migrations（マイグレーションの実行）
- 作成したマイグレーションファイルの内容をデータベースに適用する
- マイグレーションファイルの内容に応じた`CREATE TABLE`文や`ALTER TABLE`文を発行し、データベースに反映する
- 引数なしで実行すると、設定ファイルの`INSTALLED_APPS`に登録された全てのアプリケーションに対して処理を行う
- 個別のアプリケーションに限定するには、アプリケーション名を引数に設定する。<br />複数のアプリケーション名をスペース区切りで列挙することも可能

### 自作したアプリケーションにマイグレーションが適用されない場合の確認事項
1. アプリケーション設定ファイルの`INSTALLED_APPS`に登録されているか
1. アプリケーションディレクトリに`migration`ディレクトリが存在するか
1. `migration`ディレクトリに`\_\_init\_\_.py`が存在するか
1. データベースにマイグレーション履歴のレコードが残ったままになっていないか<br />履歴レコードが不整合を起こして現在のマイグレーションファイルの番号よりも進んでしまっている場合はマイグレーションが実行されない場合がある

## 11.4 マイグレーション履歴
- Djangoはマイグレーションの履歴を`django_migrations`というテーブルで管理している
  - `migrate`コマンドで適用されたマイグレーションの履歴が、マイグレーションファイルごとに1行ずつのレコードとして保存されている
  
### マイグレーション履歴の確認
以下のコマンドで、`makemigrations`で作成されたマイグレーションファイルがどこまで`migrate`で適用されているかを確認できる
~~~
python3 manage.py showmigration
~~~

出力されたマイグレーションファイル名の先頭に「**X**」がついているものは`migrate`が適用されて履歴レコードとして保存されている  
「**X**」が付いていないものはマイグレーションファイルは存在するが、`migrate`はまだ実行されていない

### マイグレーション履歴を用いたデータベースのロールバック
以下のように、`migrate`コマンドの引数にアプリケーション名とマイグレーションファイル名を指定することで、履歴を用いてデータベースの状態をロールバックすることが出来る
~~~
python3 manage.py migrate shop 0001_initial
~~~

## まとめ
- マイグレーションの目的は、データベースの準備
- マイグレーションコマンドは`makemigrations`と`migrate`の二段構え
  - `makemigrations`ではマイグレーションファイルの作成
  - `migrate`でマイグレーションファイルをデータベースに反映